# mapping NJT bus routes using Clever Devices API route XML

# global settings

In [2]:
source = 'nj'
rt = 319
direction = 0 # inbound

In [1]:
import os
import sys
sys.path.insert(os.path.join( os.path.dirname( __file__ ), '..' ))


NameError: name '__file__' is not defined

In [ ]:
# gmaps tutorial https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

In [4]:
import sys
import lib.Buses as Buses
import gmaps
import config

gmaps.configure(api_key=config.free_maps_api_key)

In [5]:
# return a list of routes, by direction

##
## I THINK WE ARE STILL GETTING BACK BOTH DIRECTIONS FOR EACH DIRECTION FROM parse_route_xml
## NEED TO GROUND TRUTH THE POINTS - MAYBE FOR A SHORTER ROUTE?
##

routes=Buses.parse_route_xml(Buses.get_xml_data(source, 'routes', route=rt))


In [6]:
# 1. split stops and points (Alex original map)

# separate figures for route points and stop points (Alex original version) - direction inbound
route_points = []
stops_points = []

for i in routes[0][direction].points:
    if isinstance(i, Buses.Route.Stop):
        stops_points.append(i)
    else:
        route_points.append(i)
            
m = gmaps.Map()

m.add_layer(gmaps.symbol_layer([(float(p.lat), float(p.lon)) for p in route_points], fill_color='green', stroke_color='green', scale=2))
m.add_layer(gmaps.symbol_layer([(float(p.lat), float(p.lon)) for p in stops_points], fill_color='red', stroke_color='red', scale=5))
m

TWFwKGNvbmZpZ3VyYXRpb249eydhcGlfa2V5JzogdSdBSXphU3lCOFNIT0VhaW9qTlZ4QWRnMjQ2NlktdWpua2UwWWZlUkUnfSwgZGF0YV9ib3VuZHM9WygzOC44Nzg0MzYwMzIyMTc4MSwgLTfigKY=


In [7]:
# 2.stops only 

# separate figures for route points and stop points (Alex original version) - direction inbound
route_points = []
stops_points = []

for i in routes[0][direction].points:
    if isinstance(i, Buses.Route.Stop):
        stops_points.append(i)
    else:
        route_points.append(i)
            
m = gmaps.Map()

#m.add_layer(gmaps.symbol_layer([(float(p.lat), float(p.lon)) for p in route_points], fill_color='green', stroke_color='green', scale=2))
m.add_layer(gmaps.symbol_layer([(float(p.lat), float(p.lon)) for p in stops_points], fill_color='red', stroke_color='red', scale=5))
m

TWFwKGNvbmZpZ3VyYXRpb249eydhcGlfa2V5JzogdSdBSXphU3lCOFNIT0VhaW9qTlZ4QWRnMjQ2NlktdWpua2UwWWZlUkUnfSwgZGF0YV9ib3VuZHM9WygzOC44Nzg0MzYwMzIyMTc4MSwgLTfigKY=


In [8]:
# 3. map - simple start-end route, no waypoints

# set up the directions-eligible API key
import config
gmaps.configure(api_key=config.paid_directions_api_key)


# separate figures for route points and stop points (Alex original version)
route_points = []
stops_points = []

for i in routes[0][direction].points:
    if isinstance(i, Buses.Route.Stop):
        stops_points.append(i)
    else:
        route_points.append(i)

end_num = (len(stops_points) - 1)
start = (float(stops_points[0].lat), float(stops_points[0].lon))
end = (float(stops_points[end_num].lat), float(stops_points[end_num].lon))

fig = gmaps.figure()
start2end = gmaps.directions_layer(start, end)
fig.add_layer(start2end)
fig



Figure(layout=FigureLayout(height=u'420px'))

In [9]:
# 4. map - waypoints map 

# set up the directions-eligible API key
import config
gmaps.configure(api_key=config.paid_directions_api_key)

# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

# separate figures for route points and stop points (modified from Alex original version)
way_points_full = []

# no longer distinguish between path stops and points from the XML
for i in routes[0][direction].points:
    way_points_full.append(i)

# slice out the waypoints for mapping
number_of_way_points=10 # user configurable up to 24 (maybe limit to 23 since we prepend the start point below)
                        # (per https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#directions-layer)



way_points_bins = list(chunks(way_points_full,(len(way_points_full)/number_of_way_points)))

# way_points_bins

In [10]:
way_points = []
way_points.append(way_points_full[0])  # add the starting point
for i in way_points_bins:
    way_points.append(i[-1])
    
way_points_tuples=[]
for wp in way_points:
    way_points_tuples.append((float(wp.lat),float(wp.lon)))

# way_points_tuples

In [11]:
# make the map 

import config
gmaps.configure(api_key=config.paid_directions_api_key)

end_num = len(way_points)-1
start = (float(way_points[0].lat), float(way_points[0].lon))
end = (float(way_points[end_num].lat), float(way_points[end_num].lon))

waypoint_map = gmaps.figure()
points_layer = gmaps.directions_layer(
        start, end, waypoints=[x for x in way_points_tuples])
waypoint_map.add_layer(points_layer)
waypoint_map

Figure(layout=FigureLayout(height=u'420px'))